In [1]:
from src.data_tools.czech_data_tools import load_czech_media_data

In [5]:
from pathlib import Path

DATA_DIR = Path('data')
czech_comments = load_czech_media_data(str(DATA_DIR / 'MediaSource'))
print(f"Loaded {len(czech_comments)} comments from {czech_comments['author'].nunique()} unique authors")

Loading files: 100%|██████████████████████████████████████████████████████| 124/124 [00:10<00:00, 11.64it/s]


Loaded 845551 comments from 66578 unique authors


In [10]:
import pandas as pd
import json

# Load annotations
annotations_df = pd.read_csv('annotations/user_labels.csv')

# Filter out uncertain labels (-1)
certain_labels_df = annotations_df[annotations_df['label'].isin([0, 1])]

# Create a dictionary of user -> label
user_labels = certain_labels_df.set_index('author')['label'].to_dict()

# Group comments by user
user_comments = {}
for user in user_labels.keys():
    # Get all comments for this user
    user_data = czech_comments[czech_comments['author'] == user]
    
    # Concatenate all comments for this user
    all_comments = ' '.join(user_data['text'].astype(str).tolist())  # replace 'text' with your actual comment column name
    
    user_comments[user] = all_comments

# Create MetaTroll format dataset
metatroll_data = []
for user, comments in user_comments.items():
    metatroll_data.append({
        'user_id': user,
        'clean_tweets_string': comments,
        'troll_domain': 'czech_social',  # single domain for all Czech data
        'encoded_label': user_labels[user]
    })

# Convert to DataFrame and save
metatroll_df = pd.DataFrame(metatroll_data)
metatroll_df.to_pickle('czech_troll_data.pkl')

In [11]:
metatroll_df.head()

,user_id,clean_tweets_string,troll_domain,encoded_label
0,Štěpán Malák,"nechce ztrapňovat ukazovat svoji neschopnost, ...",czech_social,1
1,Jan Benda,přesně Mnichov 1938 nás!!!! Elektrokrámy předr...,czech_social,1
2,Jindra Macek,Bravo Slovaci♥️ invazi Ruska UA maji cesi nejd...,czech_social,1
3,Josef Fortelný,"Jo Zelenský somrák nevděčný, víc… Okamuru stíh...",czech_social,1
4,Michal Musil,skvely hrdinove rusaci okupujou ukrajinu ukraj...,czech_social,1
